In [1]:
"""
當用戶關注、發訊息，把Json寫進txt檔，並Echo

"""

'\n當用戶關注、發訊息，把Json寫進txt檔，並Echo\n\n'

In [2]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, FollowEvent
)

import json

app = Flask(__name__)

# 取出 channelAccessToken 和 channelSecret
secretFile = json.load(open("../secretFile.txt",'r'))
channelAccessToken = secretFile['channelAccessToken']
channelSecret = secretFile['channelSecret']

line_bot_api = LineBotApi(channelAccessToken)
handler = WebhookHandler(channelSecret)

# 從 Line 的 Webhook URL 接到這裡
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

In [3]:
# 當收到 FollowEvent
@handler.add(FollowEvent)
def handle_follow(event):
    
    # 將接到的 Json 寫入 recordJsonFromLine.txt
    recordFile = open("../recordJsonFromLine.txt",'a')
    recordFile.write(str(event) + "\n")
    print(str(event))

In [4]:
# 當收到 MessageEvent, 而且是文字訊息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    # 將接到的 Json 寫入 recordJsonFromLine.txt
    recordFile = open("../recordJsonFromLine.txt",'a')
    recordFile.write(str(event) + "\n")
    print(str(event))
    
    # 回復文字訊息
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))

In [ ]:
# 啟動 Server
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


{"message": {"id": "9756055143441", "text": "\u54c8\u54c8", "type": "text"}, "replyToken": "27e36e9486c54190bca54594a791e151", "source": {"type": "user", "userId": "U3e61d72f01f625028c87e434e3a894d0"}, "timestamp": 1556203751713, "type": "message"}


172.17.0.3 - - [25/Apr/2019 13:48:15] "POST / HTTP/1.1" 200 -
